# 权重衰减——代码实现

权重衰减是最广泛使用的正则化技术之一。

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

首先，还是先生成一些人工的数据集。

$$
y = 0.05 + \sum_{i=1}^{d} 0.01x_i + \epsilon \quad \text{其中} \epsilon \sim N(0, 0.01^2)
$$


In [2]:
n_train, n_test, num_inputs, batch_size = 20, 100, 200, 5 # 特意将训练数据设置的比较小，这里是为了产生过拟合，因为输入维度有200
true_w = torch.ones((num_inputs, 1)) * 0.01
true_b = 0.05

train_data = d2l.synthetic_data(true_w, true_b, n_train)
train_iter = d2l.load_array(train_data, batch_size)

test_data = d2l.synthetic_data(true_w, true_b, n_test)
test_iter = d2l.load_array(test_data, batch_size, is_train=False)

初始化参数：

In [3]:
def init_params():
    w = torch.normal(0, 1, size=(num_inputs,1), requires_grad=True)
    b = torch.zeros(1, requires_grad=True)
    return [w,b]

权重衰减新增：L2范数惩罚项，即`||w||^2 /2`

In [4]:
def l2_penalty(w):
    return torch.sum(w.pow(2)) / 2

训练代码：